In [1]:
import pandas as pd
import ast
import json
import random
from openai import OpenAI
import re

# Загрузка данных из xlsx файла
df = pd.read_excel('webglm_full.xlsx')

In [2]:
len(df["answer"])

43579

In [3]:
def find_sources(text):
    # Регулярное выражение для поиска источников вида [число]
    pattern = r'\[\d+\]'
    sources = re.findall(pattern, text)
    return sources

In [6]:
from typing import List, Dict

def create_llama_dataset(df: pd.DataFrame) -> List[Dict]:
    dataset = []
    for _, row in df.iterrows():
        if len(find_sources(row['answer'])) > 0:
            question = row['question']
            answer = str(row['answer'])
            examples = ast.literal_eval(row["references"]) if isinstance(row['references'], str) else []
            
            dataset_item = {
                "question": question,
                "answer": answer,
                "references": examples
            }
            dataset.append(dataset_item)
    return dataset

# Применение функции к вашему датасету
llama_dataset = create_llama_dataset(df)
len(llama_dataset)

6557

In [7]:
SYSTEM_PROMPT = (
    "Ты — экспертная система Compressa RAG, "
    "предоставляющая точные и релевантные ответы на вопросы, "
    "используя только предоставленную контекстную информацию. "
    "Отвечай на русском языке."
)
REJECT_ANSW = "К сожалению, ответа на вопрос нет в упомянутых источниках"

def get_summary_prompt(context_list, question):
    context = ''
    for i, c in enumerate(context_list):
        context += f'Источник [{i+1}]:'+"\n"+c+"\n\n"

    prompt = (
        f"# Контекстная информация:\n\n{context}\n\n"
        "---\n"
        "# Инструкции:\n\n"
        "1. Дай полный и краткий ответ на вопрос, используя только информацию из контекста.\n"
        "2. Укажи номер источника в квадратных скобках после использовании фактов из него, например: [1].\n"
        "3. Каждый ответ должен содержать хотя бы одну ссылку на источник.\n"
        "4. Ссылайся на источник только если информация взята из него.\n"
        f"5. Если ответа на вопрос нет в источниках, ответь: \"{REJECT_ANSW}\".\n"
        "6. Не используй знания вне предоставленного контекста.\n\n"
        "7. Будь очень внимательна к условиям, которым твой ответ не должен противоречить. "
        "Пример: товар для доставки не должен попадать в определенную категорию.\n\n"
        "# Пример:\n\n"
        "**Контекстная информация:**\n"
        "Источник [1]:\n"
        "Всегда носите защитные очки при работе с оборудованием.\n\n"
        "Источник [2]:\n"
        "Перед началом работы проверьте оборудование на наличие повреждений.\n\n"
        "**Вопрос:** Нужно ли носить защитные очки при работе с оборудованием?\n\n"
        "**Ответ:** Да, при работе с оборудованием необходимо носить защитные очки [1].\n\n"
        "---\n"
        f"# Вопрос:\n\n{question}\n"
    )
    return prompt

def update_refs_number(answer, old_numbers, new_numbers):
    for old, new in zip(old_numbers, new_numbers):
        answer = answer.replace(f"[{old}]", f"[{new}]")
    return answer

In [6]:
main_val_json = []
for i, q in enumerate(llama_dataset[:3000]):
    main_val_json.append({
        "messages": [
            {"content": SYSTEM_PROMPT, "role": "system"},
            {"content": get_summary_prompt(q["references"], q["question"]), "role": "user"},
            {"content": q["answer"], "role": "assistant"}
        ],
        "source": "human",
        "opus_score": 10,
        "landuage": "Russian"
    })

with open("sft_train.jsonl", "w") as json_file:
    for item in main_val_json:
        json.dump(item, json_file)
        json_file.write('\n')

In [12]:
main_json = []
slice_dataset = llama_dataset[3000:3500]
max_refs = 100

if len(slice_dataset) < max_refs:
    max_refs = len(slice_dataset)
    
for i, q in enumerate(slice_dataset):
    true_refs = q["references"]
    all_refs = []
    get_refs_number = i+1
    while len(all_refs) < max_refs:       
        if get_refs_number > len(slice_dataset)-1:
            get_refs_number = 0
            
        all_refs += slice_dataset[get_refs_number]["references"]
        get_refs_number += 1

    all_refs += true_refs
    random.shuffle(all_refs)
    new_numbers = [all_refs.index(ref) + 1 for ref in true_refs]
    answer = update_refs_number(q["answer"], list(range(1, len(new_numbers)+1)), new_numbers)
    main_json.append({
        "messages": [
            {"content": SYSTEM_PROMPT, "role": "system"},
            {"content": get_summary_prompt(all_refs, q["question"]), "role": "user"},
            {"content": answer, "role": "assistant"}
        ],
        "source": "human",
        "opus_score": 10,
        "landuage": "Russian"
    })

with open("sft_rag_val.jsonl", "w") as json_file:
    for item in main_json:
        json.dump(item, json_file)
        json_file.write('\n')

In [10]:
len(main_json[1]["messages"][1]["content"])

23755

In [28]:
client = OpenAI(
    base_url="http://127.0.0.1:5000/v1",
    api_key="token-123",
)

In [44]:
num = 35
completion = client.chat.completions.create(
  model="RAG",
  messages=main_json[num]["messages"][:2]
)

print(slice_dataset[num]["question"])
print("ОТВЕТ НЕЙРОСЕТИ", completion.choices[0].message.content, "\n")
print("ОТВЕТ ИЗ ДАТАСЕТА", main_json[num]["messages"][2]["content"])

Почему люди, которые сильно пьют, становятся старше.
ОТВЕТ НЕЙРОСЕТИ Люди, которые сильно пьют, могут стать старше по нескольким причинам: их кожа может стать старше, чем она должна быть, и они могут набрать вес.Это связано с тем, что алкоголь может улучшить процесс старения в целом, заставляя людей стареть быстрее, чем обычно.Он также может оказать непосредственное влияние на определенные части тела и их психическое здоровье с возрастом.[14] Кроме того, алкоголь может также преждевременно заставлять стареть части тела, которые вы не можете видеть, например, мозг.[46] 

ОТВЕТ ИЗ ДАТАСЕТА Сильное употребление алкоголя может привести к быстрому физическому старению, включая изменения физического внешнего вида, такие как развитие серого кольца в глазу (аркус роговицы) и изменения кожи. Он также может заставлять мозг стареть, что приводит к снижению производительности в тестах исполнительной функции. Алкоголь также связан с повышенным риском возникновения медицинских проблем с возрастом, т

In [12]:
with open("sft_rag_train.jsonl", "r") as file:
    for line in file.readlines():
        data = json.loads(line)

In [17]:
data["messages"][2]

{'content': 'Если вы съели слишком много или выпили слишком много жидкости, пища может находиться в вашем желудке слишком долго и вызывать гастропарез или задержку опустошения желудка.[102] Кроме того, если вы употребляли жидкости или продукты, содержащие добавленный сахар или другие подсластители, пили слишком много алкоголя, принимали лекарства, которые вызывают задержку жидкости или имеют слабительный эффект, или из-за другого медицинского состояния, такого как гастрит, жидкость может проникать в ваш желудок из неожиданного источника.[102] Наконец, если вы съели небольшую еду и испытали разжигание, пища может перемещаться через пищеварие.',
 'role': 'assistant'}

In [18]:
def read_jsonl(file_name):
    with open(file_name, encoding="utf-8") as r:
        return [json.loads(line) for line in r]

In [20]:
data = read_jsonl("sft_rag_train.jsonl")

In [26]:
train_records = read_jsonl("sft_rag_train.jsonl")
val_records = read_jsonl("sft_rag_val.jsonl")

datasets = []
for records in (train_records, val_records):
    print(records)
    time.sleep(1)

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [27]:
len(train_records), len(val_records)

(3000, 500)

In [29]:
rec = (train_records[1], val_records[1])

In [30]:
for i in ([1,2,3,4,5,6,7,8,9], [11,22,33,44,55,66,77,88]):
    print(i)

[1, 2, 3, 4, 5, 6, 7, 8, 9]
[11, 22, 33, 44, 55, 66, 77, 88]
